In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark\\spark-3.0.1-bin-hadoop2.7'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName ('Titanic Data') \
    .getOrCreate()
spark

In [3]:
df = (spark.read
     .format("csv")
     .option("header","true")
     .load("train.csv"))

In [4]:
df.show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [5]:
from pyspark.sql.functions import col
from pyspark.sql.functions import isnull, when, count, col

dataset = df.select(col('Survived').cast('float'),
                    col('Pclass').cast('float'),
                    col('Sex'),
                    col('Age').cast('float'),
                    col('Fare').cast('float'),
                    col('Embarked')
                    )
dataset.select([count(when(isnull(c),c)).alias(c) for c in dataset.columns]).show()
dataset = dataset.replace('?', None)\
        .dropna(how='any')

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|177|   0|       2|
+--------+------+---+---+----+--------+



In [6]:
dataset.select([count(when(isnull(c),c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|  0|   0|       0|
+--------+------+---+---+----+--------+



In [7]:
dataset.show(10)

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
|     1.0|   1.0|female|35.0|   53.1|       S|
|     0.0|   3.0|  male|35.0|   8.05|       S|
|     0.0|   1.0|  male|54.0|51.8625|       S|
|     0.0|   3.0|  male| 2.0| 21.075|       S|
|     1.0|   3.0|female|27.0|11.1333|       S|
|     1.0|   2.0|female|14.0|30.0708|       C|
|     1.0|   3.0|female| 4.0|   16.7|       S|
+--------+------+------+----+-------+--------+
only showing top 10 rows



In [8]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

In [9]:
(training_data, test_data) = dataset.randomSplit([0.8,0.2], 11)
print("Nummber of train Sample : " + str(training_data.count()))
print("Nummber of test Sample : " + str(test_data.count()))

Nummber of train Sample : 562
Nummber of test Sample : 150


In [10]:
Sex_indexer = StringIndexer(inputCol='Sex', outputCol='Gender')
Embarked_indexer = StringIndexer(inputCol='Embarked', outputCol='Boarded')

inputCols = ['Pclass', 'Age', 'Fare', 'Gender', 'Boarded']
outputCol = "Features"

vector_assembler = VectorAssembler(inputCols=inputCols, outputCol=outputCol)
dt_model =  RandomForestClassifier(labelCol='Survived', featuresCol='Features')

pipeline = Pipeline(stages=[Sex_indexer, Embarked_indexer, vector_assembler, dt_model])

final_pipeline = pipeline.fit(training_data)

test_predictions_from_pipeline = final_pipeline.transform(test_data)

test_predictions_from_pipeline.show(500, truncate=False)

+--------+------+------+----+--------+--------+------+-------+--------------------------------------------------+----------------------------------------+-----------------------------------------+----------+
|Survived|Pclass|Sex   |Age |Fare    |Embarked|Gender|Boarded|Features                                          |rawPrediction                           |probability                              |prediction|
+--------+------+------+----+--------+--------+------+-------+--------------------------------------------------+----------------------------------------+-----------------------------------------+----------+
|0.0     |1.0   |male  |19.0|263.0   |S       |0.0   |0.0    |[1.0,19.0,263.0,0.0,0.0]                          |[12.960226034858389,7.039773965141613]  |[0.6480113017429194,0.3519886982570807]  |0.0       |
|0.0     |1.0   |male  |21.0|77.2875 |S       |0.0   |0.0    |[1.0,21.0,77.2874984741211,0.0,0.0]               |[10.836898485215048,9.163101514784953]  |[0.54184492426